# CU Icecream Data
- CU 웹사이트 특징:
    1. 더보기 버튼이 총 7번 있다
    2. 브랜드 명은 ')' 앞에, 이름은 뒤에 있다
    3. HEYROO와 delaffe만 브랜드명과 이름이 ')'구분 없이 붙어있다

#### 1. 필요한 패키지 참조

In [2]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time

from bs4 import BeautifulSoup
from pandas import DataFrame

import pymysql

#### 2. Chrome Browser 가동

In [3]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

#### 3. CU 아이스크림 페이지로 이동

In [4]:
driver.get("https://cu.bgfretail.com/product/product.do?category=product&depth2=4&depth3=4")
# 1초간 대기
time.sleep(1)

#### 4. 더보기 버튼 누르기 (총 7번)

In [5]:
a = 0
for a in range(7):
    if a < 7:
        more_button = WebDriverWait(driver, 3).until(
            lambda x: x.find_element(By.CSS_SELECTOR, ".prodListBtn-w")
        )
        more_button.click()
        # 5초간 대기
        time.sleep(5)
        a += 1

#### 5. 내용 수집

##### 1) 모든 정보 수집할 리스트 생성 및 아이스크림의 정보가 표시되는 html&css 추출

In [6]:
icecream_data = []

soup = BeautifulSoup(driver.page_source)
icecreamList = soup.select(".prod_wrap")

##### 2) 추출된 아이스크림 목록 수 만큼 반복하며 이름, 브랜드명, 가격 가져온 후 결과 병합

In [7]:
for i in icecreamList:
    # 이름
    name = i.select(".name")
    if name:
        nameValue = name[0].text.strip()
        if ')' in nameValue:
            indexBeforeName = nameValue.index(')')
            nameValue = nameValue[indexBeforeName + 1:]
        elif 'delaffe' in nameValue:
            nameValue = nameValue[7:]
        elif 'HEYROO' in nameValue:
            nameValue = nameValue[6:]
        else:
            nameValue = nameValue
    else: 
        nameValue = None
    
    # 브랜드명
    brand = i.select(".name")
    if brand:
        brandValue = brand[0].text.strip()
        if ')' in brandValue:
            indexBeforeName = brandValue.index(')')
            brandValue = brandValue[:indexBeforeName]
        elif 'delaffe' in brandValue:
            brandValue = 'delaffe'
        elif 'HEYROO' in brandValue:
            brandValue = 'HEYROO'
        else:
            brandValue = None
    else:
        brandValue = None

    # 가격
    price = i.select(".price")
    if price:
        priceValue = int(price[0].text.strip().replace('원', '').replace(',', ''))
    else:
        priceValue = None

    # 결과 병합
    resultDic = {'이름': nameValue, 
                '브랜드': brandValue,
                '가격(원)': priceValue}
    icecream_data.append(resultDic)

#### 6. Excel 파일로 수집한 내용 저장

In [8]:
df = DataFrame(icecream_data)
df.to_excel("icecream_list_cu.xlsx")

In [16]:
len(icecream_data)

307

#### 7. MySQL에 저장 - pymysql 사용
##### 데이터 베이스에 접속

In [10]:
dbcon = pymysql.connect(host="127.0.0.1",   # 서버주소 - "127.0.0.1" 또는 "localhost"
                        port = 3406,        # 포트번호
                        user = "root",      # 계정이름
                        password = "1234",  # 비밀번호
                        db = "icecream",    # 데이터베이스 이름
                        charset = "utf8"    # 인코딩
                        )

##### 커서 객체 생성

In [11]:
cursor = dbcon.cursor()

##### 데이터 입력

In [13]:
웹페이지_이름 = ['cu_icecream']

for 웹페이지 in 웹페이지_이름:
    # 테이블 생성
    create_table = "CREATE TABLE %s (id INT auto_increment, name VARCHAR(10), brand VARCHAR(10), price INT, PRIMARY KEY (id))" % 웹페이지
    cursor.execute(create_table)         # 쿼리문 실행

    # 각 테이블 (CU, EMART)에 해당 아이스크림 정보 입력
    for 아이스크림 in range(len(icecream_data)):
        이름 = icecream_data[아이스크림]['이름']
        브랜드 = icecream_data[아이스크림]['브랜드']
        가격 = icecream_data[아이스크림]['가격(원)']
        insert = "INSERT INTO %s (name, brand, price) VALUES ('%s', '%s', %d)" % (웹페이지, 이름, 브랜드, 가격)
        cursor.execute(insert)         # 쿼리문 실행

##### 데이터 조회/확인

In [14]:
for 웹페이지 in 웹페이지_이름:
    조회 = "SELECT * FROM %s" % 웹페이지
    cursor.execute(조회)         # 쿼리문 실행
    result = cursor.fetchall()  # 데이터 반환받기
    df = DataFrame(result)
    df.set_index(0, inplace= True)
    df.index.name = 'id'
    df.rename(columns = {1: '이름', 2: '브랜드', 3: '가격(원)'}, inplace=True)

In [15]:
df

,이름,브랜드,가격(원)
id,,,
1,녹차미니컵,나뚜루,4800
2,녹차바,나뚜루,4800
3,딸기미니컵,나뚜루,4800
4,바닐라아몬드바,나뚜루,4800
5,초코미니컵,나뚜루,4800
...,...,...,...
303,초코미니컵,나뚜루,4800
304,바닐라아몬드바,나뚜루,4800
305,딸기미니컵,나뚜루,4800
